In [ ]:
# Verify we have CUDA, if not do one or all of the following:
# 1. Change your runtime to GPU using Runtime => Change runtime type => GPU
# 2. Sometimes CUDA is just not here, if that happens reset the runtime using Runtime => Restart runtime

! lsb_release -a
! uname -m 
! nvcc --version
! nvidia-smi

# Tensorflow Setup

In [ ]:
# We need to uninstall the tensorflow that came with this session, as SciSharp.TensorFlow.Redist-Linux-GPU requires 2.3.1

# Uninstall the tensorflow version that came with this session
! pip uninstall --yes tensorflow

# Install tensorflow 2.3.1 (Do not restart the runtime if prompted)
! pip install tensorflow==2.3.1

# Install tensorflow-gpu 2.3.1 (Do not restart the runtime if prompted)
! pip install tensorflow-gpu==2.3.1

In [ ]:
# Verify that the version of tensorflow we're expecting is installed. If 2.3.1 is not listed, you can now restart the runtime, as
# requested in the previous step.

# Colab should launch with tensorflow 2.X as the default, but we ensure its set as 2.X just in case
%tensorflow_version 2.x

# Display the version pip installed and its location
! pip show tensorflow

# Display the version pip installed, as well as GPUs that are available
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
# Install the two additional packages required as per the SciSharp docs: https://github.com/SciSharp/TensorFlow.NET/tree/master/tensorflowlib

# libc6-dev should already be installed
#! sudo apt install libc6-dev 
! sudo apt install libgdiplus

In [ ]:
# Lastly we pull down the 'libtensorflow' libraries required via the SciSharp docs: https://github.com/SciSharp/TensorFlow.NET/tree/master/tensorflowlib
# We used the Tensorflow docs located here for help: https://www.tensorflow.org/install/lang_c

# Pull the libtensorflow.so files
! wget -q https://storage.googleapis.com/tensorflow/libtensorflow/libtensorflow-gpu-linux-x86_64-2.3.1.tar.gz

# Install the files where they are expected
! sudo tar -C /usr/local -xzf libtensorflow-gpu-linux-x86_64-2.3.1.tar.gz

# Run 'ldconfig' to configure the linker
! sudo ldconfig

# Set the appropriate environment variables and paths so the libtensorflow libraries can be found
%env LIBRARY_PATH=$LIBRARY_PATH:/usr/local
%env LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local

# Install .NET 3.1 & 6.0 and ML.NET

In [ ]:
# Install .NET Core 3.1 & 6.0 & ML.NET
! wget -q https://packages.microsoft.com/config/ubuntu/18.04/packages-microsoft-prod.deb -O packages-microsoft-prod.deb \
&& dpkg -i packages-microsoft-prod.deb \
&& add-apt-repository universe \
&& apt-get update \
&& apt-get install apt-transport-https \
&& apt-get update \
&& apt-get install dotnet-sdk-3.1 \
&& apt-get install dotnet-sdk-6.0 \
&& dotnet tool install -g mlnet

# Staging Our Data

In [ ]:
# Mount our Drive
from google.colab import drive
drive.mount('/content/drive')

In [8]:
# Unzip the transformed images
! unzip -qq /content/drive/MyDrive/Catovera/fashion-product-images-dataset/transformed_images.zip -d unzipped_images

In [ ]:
# Verify the training data came over, in this case >= 178139 images
! ls unzipped_images/unzipped_images/Images/ | wc -l

In [ ]:
# in order to upload a CSV we need to upload via files.upload to avoid Drive converting the file
#from google.colab import files
#uploaded = files.upload()

# copy the traning csv into the images folder
#! cp -a all_data.csv unzipped_images/unzipped_images/Images/

In [33]:
! cp -a /content/drive/MyDrive/Catovera/ml-color-model/all_data_select.csv unzipped_images/unzipped_images/Images/

# renaming the file here until the code is updated to have that passed in
! mv unzipped_images/unzipped_images/Images/all_data_select.csv unzipped_images/unzipped_images/Images/training_data.csv

# Stage Our Code

In [12]:
# Make a "build-ouput" directory to hold compiled training code
! mkdir build-output

In [ ]:
# repeatable remove folder, then copy it back in, modify permissions so we can run it
! rm -rf build-output/ColabOutput
! cp -a /content/drive/MyDrive/Catovera/ml-color-model/ColabOutput/ build-output/
! chmod -R 777 build-output/ColabOutput/

In [ ]:
# verify the files are copied over, should be 3 files/folders: App  App.pdb  Data
! ls build-output/ColabOutput/

# Train and Save Model

In [ ]:
# train the model
! ./build-output/ColabOutput/App --train --folder="unzipped_images/unzipped_images/Images"

In [70]:
# move the final model out to the ml-color-model folder
! cp -a build-output/ColabOutput/color-classifier.mdl /content/drive/MyDrive/Catovera/ml-color-model/12-13-21-22-53-00-color-classifier.mdl

In [ ]:
! ls /content/drive/MyDrive/Catovera/ml-color-model/